In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Credit to VK_ds

**Load the Dataset**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
train_df=pd.read_csv('../input/train.csv')
test_df=pd.read_csv('../input/test.csv')

Data Dictionary
Variable Definition Key

survival Survival 0 = No, 1 = Yes

pclass Ticket class 1 = 1st, 2 = 2nd, 3 = 3rd

sex Male or Female

Age Age in years

sibsp # of siblings / spouses aboard the Titanic

parch # of parents / children aboard the Titanic

ticket Ticket number

fare Passenger fare

cabin Cabin number

embarked Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton

Variable Notes pclass: A proxy for socio-economic status (SES) 1st = Upper 2nd = Middle 3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...

Sibling = brother, sister, stepbrother, stepsister

Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...

Parent = mother, father

Child = daughter, son, stepdaughter, stepson

Some children travelled only with a nanny, therefore parch=0 for them

In [ ]:
def missingdata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms = ms[ms['Percent'] > 0]
    f, ax = plt.subplots(figsize=(8,6))
    plt.xticks(rotation='90')
    fig = sns.barplot(ms.index, ms['Percent'], color='green', alpha=0.8)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('Percent of missing values', fontsize=15)
    plt.title('Percent missing data by feature', fontsize=15)
    return ms

In [ ]:
missingdata(train_df)

In [ ]:
missingdata(test_df)

Filling missing values

In [ ]:
test_df['Age'].mean()

In [ ]:
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace = True)

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

Cabin feature has more than 75% of missing data in both Test and Train data.
So we are removing Cabin feature.

In [ ]:
drop_column = ['Cabin']
train_df.drop(drop_column, axis=1, inplace=True)
test_df.drop(drop_column, axis=1, inplace=True)

Both the test and train Age feature contains more than 15% of missing data.
So we fill in the missing data with median.

In [ ]:
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)
train_df['Age'].fillna(test_df['Age'].median(), inplace=True)

In [ ]:
print('check the nan value in train data')
print(train_df.isnull().sum())
print('___'*30)
print('check the nan value in test data')
print(test_df.isnull().sum())

**Feature Engineering**

In [ ]:
# combine test and train as single to apply function
all_data = [train_df, test_df]

In [ ]:
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in all_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

In [ ]:
import re
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search('([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

# Create a new feature Title, containing the titles of passenger names
for dataset in all_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

# Group all non-common titles into one single grouping 'Rare'
for dataset in all_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                                 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [ ]:
# create bin for age features
for dataset in all_data:
    dataset['Age_bin'] = pd.cut(dataset['Age'], bins=[0,12,20,40,120], labels=['Children','Teenage','Adult','Elder'])

In [ ]:
# create bin for fare features
for dataset in all_data:
    dataset['Fare_bin'] = pd.cut(dataset['Fare'], bins=[0,7.91,14.45,31,120], labels=['Low_fare','Median_fare','Average_fare','High_fare'])

In [ ]:
# for our reference, making a copy of both DataSet.
# Start working on dataset copy.
traindf = train_df
testdf = test_df

In [ ]:
all_data = [traindf, testdf]

In [ ]:
for dataset in all_data:
    drop_column  = ['Age', 'Fare', 'Name', 'Ticket']
    dataset.drop(drop_column, axis=1, inplace=True)

In [ ]:
drop_column = ['PassengerId']
traindf.drop(drop_column, axis=1, inplace=True)

now everything almost ready only one step we convert the categorical features in numberical by using dummy variable

In [ ]:
testdf.head(2)

In [ ]:
traindf = pd.get_dummies(traindf, columns = ['Sex','Title','Age_bin','Embarked','Fare_bin'],
                         prefix=['Sex','Title','Age_type','Em_type','Fare_type'])

In [ ]:
testdf = pd.get_dummies(testdf, columns = ['Sex','Title','Age_bin','Embarked','Fare_bin'],
                        prefix=['Sex','Title','Age_type','Em_type','Fare_type'])

In [ ]:
testdf.head()

Correlation between the Features

In [ ]:
sns.heatmap(traindf.corr(), annot=True, cmap='RdYlGn',linewidths=0.2) #data.corr() --> correlation matrix
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()

**Pairplots**

Finally let us generate some pairplots to observe the distribution of data from one feature to the other. Once again we use Seaborn to help us.

In [ ]:
g = sns.pairplot(data=train_df, hue='Survived', palette = 'seismic',
                 size=1.2,diag_kind = 'kde',diag_kws=dict(shade=True),plot_kws=dict(s=10) )
g.set(xticklabels=[])

**Model**

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix
all_features = traindf.drop("Survived", axis=1)
Targeted_feature = traindf['Survived']
X_train, X_test, y_train, y_test = train_test_split(all_features,Targeted_feature, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train,y_train)
prediction_lr =  model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of Logistic Regression is', round(accuracy_score(prediction_lr,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_lr = cross_val_score(model,all_features,Targeted_feature,cv=10,scoring='accuracy')
print('The cross validated score for Logistic Regression is:',round(result_lr.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**Random Forest Classifier**

In [ ]:
from  sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(criterion='gini', n_estimators=700, min_samples_split=10,min_samples_leaf=1,max_features='auto', oob_score=True,
                              random_state=1,n_jobs=-1)
model.fit(X_train,y_train)
prediction_rm = model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of Random Forest Classifier is', round(accuracy_score(prediction_rm,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_rm = cross_val_score(model,all_features,Targeted_feature,cv=10)
print('The cross validated score for Random Forest Classifier is:',round(result_rm.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**Support Vector Machines**

In [ ]:
from sklearn.svm import SVC, LinearSVC

model = SVC()
model.fit(X_train,y_train)
prediction_svm = model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of Support Vector Machines Classifier is', round(accuracy_score(prediction_svm,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_svm = cross_val_score(model,all_features,Targeted_feature,cv=10)
print('The cross validated score for Support Vector Machines Classifier is:',round(result_svm.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**KNN Classifer**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 4)
model.fit(X_train,y_train)
prediction_knn = model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of KNN Classifier is', round(accuracy_score(prediction_knn,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_knn = cross_val_score(model,all_features,Targeted_feature,cv=10)
print('The cross validated score for KNN Classifier is:',round(result_knn.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**Gaussian Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train,y_train)
prediction_GNB = model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of Gaussian Naive Bayes Classifier is', round(accuracy_score(prediction_GNB,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_GNB = cross_val_score(model,all_features,Targeted_feature,cv=10)
print('The cross validated score for Gaussian Naive Bayes Classifier is:',round(result_GNB.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = RandomForestClassifier(criterion='gini', min_samples_split=10,min_samples_leaf=1,max_features='auto')
model.fit(X_train,y_train)
prediction_tree = model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of Decision Tree Classifier is', round(accuracy_score(prediction_tree,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_tree = cross_val_score(model,all_features,Targeted_feature,cv=10,scoring='accuracy')
print('The cross validated score for Decision Tree Classifier is:',round(result_tree.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**Ada Boost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier()
model.fit(X_train,y_train)
prediction_adb = model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of Ada Boost Classifier is', round(accuracy_score(prediction_adb,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_adb = cross_val_score(model,all_features,Targeted_feature,cv=10,scoring='accuracy')
print('The cross validated score for Decision Tree Classifier is:',round(result_adb.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**Linear Discriminant Analysis**

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis()
model.fit(X_train,y_train)
prediction_lda = model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of Linear Discriminant Analysis Classifier is', round(accuracy_score(prediction_lda,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_lda = cross_val_score(model,all_features,Targeted_feature,cv=10,scoring='accuracy')
print('The cross validated score for Decision Tree Classifier is:',round(result_lda.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**Gradient Boosting/Descent Classifier**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train,y_train)
prediction_gbc = model.predict(X_test)
print('-----------------Accuracy of the model-----------------')
print('The accuracy of Gradient Boosting Classifier is', round(accuracy_score(prediction_gbc,y_test)*100,2))
kfold = KFold(n_splits=10,random_state=22)  # k=10, split the data into 10 equal parts
result_gbc = cross_val_score(model,all_features,Targeted_feature,cv=10,scoring='accuracy')
print('The cross validated score for Decision Tree Classifier is:',round(result_gbc.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)
sns.heatmap(confusion_matrix(Targeted_feature,y_pred),annot=True,fmt='3.0f',cmap='summer')
plt.title('Confusion_matrix',y=1.05,size=15)

**Model Evaluation**
We can now rank our evaluation of all the models to choose the best for our problem.

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 'Random Forest', 'Naive Bayes', 'AdaBoostClassifier', 'Gradient Decent',
             'Linear Discriminant Analysis', 'Decision Tree'],
    'Score': [result_svm.mean(), result_knn.mean(), result_lr.mean(), result_rm.mean(), result_GNB.mean(), result_adb.mean(), result_gbc.mean(),
             result_lda.mean(), result_tree.mean()]})
models.sort_values(by='Score',ascending=False)

By looking at all the matrices, Random Forest & SVM classifer has a higher chance in correctly predicting dead passengers.

**Hyper-Parameters Tuning**

In [ ]:
train_X = traindf.drop('Survived', axis=1)
train_Y = traindf['Survived']
test_X = testdf.drop('PassengerId', axis=1).copy()
train_X.shape, train_Y.shape, test_X.shape

In [ ]:
# Gradient boosting tuning
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
param_grid = {'loss' : ['deviance'],
              'n_estimators' : [100,200,300,400],
              'learning_rate' : [0.1,0.05,0.01,0.001],
              'max_depth' : [4,8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3,0.2,0.1]
}

modelf = GridSearchCV(model,param_grid = param_grid, cv=kfold, scoring='accuracy', n_jobs=4, verbose=1)

modelf.fit(train_X,train_Y)
modelf.best_score_
modelf.best_estimator_

In [ ]:
modelf.best_score_

In [ ]:
# Random Forest Classifier Parameters Tuning
model = RandomForestClassifier()
n_estim = range(100,1000,100)

# Search grid for optimal parameters
param_grid = {'n_estimators' :n_estim}

model_rf = GridSearchCV(model, param_grid = param_grid, cv=5, scoring='accuracy', n_jobs=4, verbose=1)
model_rf.fit(train_X,train_Y)

print(model_rf.best_score_)
model_rf.best_estimator_

In [ ]:
model = LinearDiscriminantAnalysis()
param_grid = {'tol':[0.001,0.01,.1,.2]}

model = GridSearchCV(model,param_grid, cv=5, scoring='accuracy',n_jobs=4,verbose=1)
model.fit(train_X, train_Y)
print(model.best_score_)
model.best_estimator_

In [ ]:
model = SVC()
param_grid = {'kernel':['rbf','linear'],
              'gamma':[0.001,0.01,0.1,1],
              'C':[1,10,50,100,200,300,1000]}

model_svm = GridSearchCV(model,param_grid = param_grid, cv=5, scoring='accuracy', n_jobs=4, verbose=1)
model_svm.fit(train_X,train_Y)
print(model_svm.best_estimator_)
print(modelsvm.best_score_)

Apply the Estimator which got from parameter tuning of Random forest

In [ ]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
random_forest.fit(train_X,train_Y)
Y_pred_rf = random_forest.predict(test_X)
random_forest.score(train_X,train_Y)
acc_random_forest = round(random_forest.score(train_X,train_Y)*100,2)

print('Important features')
pd.Series(random_forest.feature_importances_,train_X.columns).sort_values(ascending=True).plot.barh(width=0.8)
print('--'*30)
print(acc_random_forest)

In [ ]:
submission = pd.DataFrame({
                'PassengerID': test_df['PassengerId'],
                'Survived' : Y_pred_rf
})
submission.to_csv('Titanic_90%_accuracy.csv', index = False)